<img src="https://i.imgur.com/OSp7FVO.png"/>

# Web Scraping Caraje

# Ranking por riesgo de Proyecto, filtrado por indirectas (contrata)

In [2]:
pip install selenium beautifulsoup4 pandas


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB 487.6 kB/s eta 0:00:20
   ---------------------------------------- 0.1/9.5 MB 744.7 kB/s eta 0:00:13
    --------------------------------------- 0.1/9.5 MB 782.7 kB/s eta 0:00:12
    --------------------------------------- 0.2/9.5 MB 857.5 kB/s eta 0:00:11
    --------------------------------------- 0.2/9.5 MB 871.5 kB/s eta 0:00:11
   - -------------------------------------- 0.2/9.5 MB 835.2 kB/s eta 0:00:12
   - -------------------------------------- 0.3/9.5 MB 874.1 kB/s eta 0:00:11
   - -------------------------------------- 0.3/9.5 MB 901.1 kB/s eta 0:00:11
   - -------------------------------------- 0.4/9.5 MB 911.8 kB/s eta 0:00:10
   - -------------------------------------- 0.5/9.5 MB 960.7 kB/s eta 0:00:10
   -- ------------------------------------- 0.5/9.5 MB 981.7 kB/s eta 0:00:10
   -

# Por fin caraje, código definitivo

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import pandas as pd

# Crea el objeto webdriver
driver = webdriver.Chrome()

# Abre la página web
driver.get('https://appbp.contraloria.gob.pe/iri/wfrm/irip/indices_riesgos_v2.aspx#panel4')

# Espera a que la página cargue completamente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "RankingProyecto")))

# Haz clic en el elemento
driver.find_element(By.XPATH, '//*[@id="RankingProyecto"]').click()

# Espera a que la nueva página cargue completamente
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlModalidad")))

# Selecciona "INDIRECTA" en el menú desplegable "Modalidad de Ejecución"
modalidad_ejecucion = Select(driver.find_element(By.XPATH, '//*[@id="ddlModalidad"]'))
modalidad_ejecucion.select_by_visible_text('INDIRECTA')

# Haz clic en el botón "BUSCAR"
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnBuscar"]'))).click()

# Espera a que la tabla "GridProyectos" esté presente en el DOM
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "GridProyectos")))

# Inicializa una lista para almacenar los DataFrames de cada página
dfs = []

# Itera sobre las páginas
for _ in range(3188):

    # Obtiene el código HTML de la página
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Encuentra la tabla y extrae los datos
    table = soup.find('table', {'id': 'GridProyectos'})
    headers = []
    header_row = table.find('thead').find('tr')
    header_cols = header_row.find_all('th')
    for header_col in header_cols[:-1]:  # Excluye la última columna
        headers.append(header_col.text.strip())

    data = []
    rows = table.find_all('tr')
    rows = rows[1:]  # Salta la primera fila vacía
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols[:-1]]  # Excluye la última columna
        data.append(cols)

    # Crea un DataFrame para esta página
    df = pd.DataFrame(data, columns=headers)

    # Agrega el DataFrame a la lista
    dfs.append(df)

    # Verifica si el botón "Siguiente" está disponible
    if not driver.find_element(By.XPATH, '//*[@id="GridProyectos_next"]').get_attribute("disabled"):
        # Haz clic en el botón "Siguiente" usando execute_script
        driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, '//*[@id="GridProyectos_next"]'))

        # Espera a que la nueva página cargue completamente
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "GridProyectos")))
    else:
        # Si el botón "Siguiente" no está disponible, significa que has llegado a la última página
        break

# Cierra el navegador
driver.quit()

# Concatena los DataFrames de cada página en un solo DataFrame
df = pd.concat(dfs, ignore_index=True)

# Imprime los datos
print(df.tail(30))

      CUI/SNIP  DEPARTAMENTO  \
95587  2614151         JUNIN   
95588  2613929   LA LIBERTAD   
95589  2612413      AREQUIPA   
95590  2611067         CUSCO   
95591  2609809      AYACUCHO   
95592  2608451   LA LIBERTAD   
95593  2608040   LA LIBERTAD   
95594  2606665         TACNA   
95595  2604972       UCAYALI   
95596  2601065       HUANUCO   
95597  2591344     CAJAMARCA   
95598  2574269          PUNO   
95599  2573043    LAMBAYEQUE   
95600  2570174         JUNIN   
95601  2569677       HUANUCO   
95602  2567227        ANCASH   
95603  2560395      AMAZONAS   
95604  2559918        ANCASH   
95605  2556908        ANCASH   
95606  2550902          LIMA   
95607  2547164           ICA   
95608  2547011    LAMBAYEQUE   
95609  2541098          LIMA   
95610  2547011    LAMBAYEQUE   
95611  2541098          LIMA   
95612  2588057   LA LIBERTAD   
95613  2604970       UCAYALI   
95614  2590617      AYACUCHO   
95615  2572718  HUANCAVELICA   
95616  2586461         PASCO   

       

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95617 entries, 0 to 95616
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   CUI/SNIP             95617 non-null  object
 1   DEPARTAMENTO         95617 non-null  object
 2   ENTIDAD EJECUTORA    95617 non-null  object
 3   NOMBRE DEL PROYECTO  95617 non-null  object
 4   Puntaje              95617 non-null  object
 5   FUNCIÓN              95617 non-null  object
 6   MARCO                95617 non-null  object
dtypes: object(7)
memory usage: 5.1+ MB
None


In [11]:
df.to_csv('ranking_riesgo_proyectos_INDIRECTA.csv', index=False)